# Step 6: Filter out incoherent matches

In [ ]:
import os
from functools import partial
import numpy as np
from phathom import plotting
from phathom.registration import registration as reg
from phathom.registration.pcloud import estimate_affine, register_pts
from phathom.utils import pickle_save

In [ ]:
working_dir = '/home/jswaney/coregistration'
voxel_size = (2.0, 1.6, 1.6)  # z, y, x

In [ ]:
# Load blobs
fixed_pts_path = 'fixed_blobs.npy'
moving_pts_path = 'moving_blobs_1200.npy'

fixed_pts = np.load(os.path.join(working_dir, fixed_pts_path))
moving_pts = np.load(os.path.join(working_dirmoving_pts_path)
print('keypoints:', fixed_pts.shape, moving_pts.shape)

In [ ]:
# Load matches
idx_fixed_path = 'idx_fixed_180_2fdist.npy'
idx_moving_path = 'idx_moving_180_2fdist.npy'

idx_fixed = np.load(os.path.join(working_dir, idx_fixed_path))
idx_moving = np.load(os.path.joing(working_dir, idx_moving_path))
print('# matches loaded:', idx_fixed.shape[0])

In [ ]:
# Extract matching coordinates
fixed_keypts = fixed_pts[idx_fixed]
moving_keypts = moving_pts[idx_moving]

In [ ]:
# Convert to um
fixed_keypts_um = fixed_keypts * np.asarray(voxel_size)
moving_keypts_um = moving_keypts * np.asarray(voxel_size)

In [ ]:
# Show starting coordinate similarity
plotting.plot_correspondence(fixed_keypts_um, moving_keypts_um)

In [ ]:
# Show starting residuals
starting_residuals = reg.match_distance(fixed_keypts_um,
                                        moving_keypts_um)
plotting.plot_hist(starting_residuals)
print('Starting ave. distance [um]:', starting_residuals.mean())

In [ ]:
# Use RANSAC to estimate affine transformation
resid_thresh = None
min_samples = 30

ransac, ransac_inliers = estimate_affine(fixed_keypts_um,
                                         moving_keypts_um,
                                         'ransac',
                                         min_samples,
                                         resid_thresh)

In [ ]:
# Apply affine transformation
ransac_keypts_um = register_pts(fixed_keypts_um, ransac)

In [ ]:
# Show ransac coordinate similarity
plotting.plot_correspondence(ransac_keypts_um, moving_keypts_um)

In [ ]:
# Show residuals after ransac
ransac_residuals = reg.match_distance(ransac_keypts_um,
                                      moving_keypts_um)
plotting.plot_hist(ransac_residuals)
print('RANSAC ave. distance [um]:', ransac_residuals.mean())

In [ ]:
# Filter on distance
max_distance = 80

inlier_idx = np.where(nonrigid_residuals < max_distance)
fixed_keypts_dist = fixed_keypts[inlier_idx]
moving_keypts_dist = moving_keypts[inlier_idx]
fixed_keypts_dist_um = fixed_keypts_um[inlier_idx]
moving_keypts_dist_um = moving_keypts_um[inlier_idx]

In [ ]:
# Use sklearn for faster affine transformation, without outliers
degree = 1

model = reg.fit_polynomial_transform(fixed_keypts_dist,
                                     moving_keypts_dist,
                                     degree)
model_z, model_y, model_x = model
affine_transformation = partial(reg.polynomial_transform, 
                                degree=degree,
                                model_z=model_z, 
                                model_y=model_y,
                                model_x=model_x)

In [ ]:
# Transform the fixed keypoints in pixel units
affine_keypts = affine_transformation(pts=fixed_keypts)
affine_keypts_dist = affine_keypts[inlier_idx]

In [ ]:
# Convert to micron
affine_keypts_um = affine_keypts * np.asarray(voxel_size)
affine_keypts_dist_um = affine_keypts_um[inlier_idx]

In [ ]:
# Show affine coordinate similarity
plotting.plot_correspondence(affine_keypts_um, moving_keypts_um)

In [ ]:
# Show residuals after affine
affine_residuals = reg.match_distance(affine_keypts_um,
                                      moving_keypts_um)
plotting.plot_hist(affine_residuals)
print('Affine ave. distance [um]:', affine_residuals.mean())

In [ ]:
# Calculate displacement coherence
n_neighbors = 3

coherences = reg.coherence(n_neighbors, 
                           affine_keypts_dist_um, 
                           moving_keypts_dist_um)
plotting.plot_hist(coherences, bins=512, xlim=[0, 0.8], ylim=[0, 400])
print('Average coherence: {}'.format(coherences.mean()))

In [ ]:
# Filter out incoherent matches
min_coherence = 0.98

coherent_idx = np.where(coherence > min_coherence)
print('Found {} outliers'.format(len(coherence)-len(coherent_idx[0])))

fixed_keypts_coherent = fixed_keypts_dist[coherent_idx]
affine_keypts_coherent = affine_keypts_dist[coherent_idx]
moving_keypts_coherent = moving_keypts_dist[coherent_idx]

fixed_keypts_coherent_um = fixed_keypts__dist_um[coherent_idx]
affine_keypts_coherent_um = affine_keypts_dist_um[coherent_idx]
moving_keypts_coherent_um = moving_keypts__dist_um[coherent_idx]

In [ ]:
# Plot filtered matches
plotting.plot_pts(affine_keypts_coherent_um,
                  moving_keypts_coherent_um,
                  0.05,
                  affine_keypts_coherent_um,
                  moving_keypts_coherent_um)

In [ ]:
# Save the affine transformation
affine_path = 'affine_transformation.pkl'

pickle_save(os.path.join(working_dir, affine_path), 
            affine_transformation)

In [ ]:
# Save the filtered match coordinates
fixed_keypts_path = 'fixed_keypts.npy'
affine_keypts_path = 'affine_keypts.npy'
moving_keypts_path = 'moving_keypts.npy'

np.save(os.path.join(working_dir, fixed_keypts_path), 
        fixed_keypts_coherent)
np.save(os.path.join(working_dir, affine_keypts_path), 
        affine_keypts_coherent)
np.save(os.path.join(working_dir, moving_keypts_path), 
        moving_keypts_coherent)